In [1]:
import keras
import numpy as np
import tensorflow as tf
from keras import optimizers
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras.callbacks import LearningRateScheduler, TensorBoard
import matplotlib.pyplot as plt
from keras.initializers import he_normal
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

from keras import backend

print(keras.__version__)
weight_decay= 0.0001
batch_size    = 128
epochs        = 200
iterations    = 391
num_classes   = 10
dropout = 0.4


def build_model():
  model= Sequential()

  model.add(Conv2D(192,(5,5),padding='same',use_bias=False,kernel_initializer=he_normal(),activation='relu',input_shape=(32,32,3),kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(Conv2D(192,(1,1),padding='same',use_bias=False,kernel_initializer=he_normal(),activation='relu',kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(Conv2D(192,(1,1),padding='same',use_bias=False,kernel_initializer=he_normal(),activation='relu',kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(AveragePooling2D(pool_size=(3, 3),strides=(2,2),padding = 'same'))
  
  
  model.add(Conv2D(192,(5,5),padding='same',use_bias=False,kernel_initializer=he_normal(),activation='relu',kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(Dropout(0.4))
  model.add(Conv2D(192,(1,1),padding='same',use_bias=False,kernel_initializer=he_normal(),activation='relu',kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(Conv2D(192,(1,1),padding='same',use_bias=False,kernel_initializer=he_normal(),activation='relu',kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(AveragePooling2D(pool_size=(3, 3),strides=(2,2),padding = 'same'))
  model.add(Dropout(0.4))
  
  model.add(Conv2D(192,(5,5),padding='same',use_bias=False,kernel_initializer=he_normal(),activation='relu',kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(Dropout(0.4))
  model.add(Conv2D(192,(1,1),padding='same',use_bias=False,kernel_initializer=he_normal(),activation='relu',kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(Conv2D(10,(1,1),padding='same',use_bias=False,kernel_initializer=he_normal(),activation='relu',kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(GlobalAveragePooling2D())
  model.add(Activation('softmax'))

  opt = optimizers.SGD(momentum=0.9)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

#load data
(x_train,y_train),(x_test,y_test)=cifar10.load_data()
print(x_train.shape)
print(x_test.shape)
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
m=np.max(x_train)
x_train=(x_train)/m
x_test=(x_test)/m


# (x_train,y_train),(x_test,y_test)=cifar10.load_data()
# y_train = np_utils.to_categorical(y_train, 10)
# y_test = np_utils.to_categorical(y_test, 10)
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# mean = [125.307, 122.95, 113.865]
# std  = [62.9932, 62.0887, 66.7048]        
# for i in range(3):
#      x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
#      x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]

def scheduler(epoch):
    if epoch <= 80:
        return 0.05
    if epoch <= 121:
        return 0.005
    if epoch <= 200:    
        return 0.0005
    # return 0.0004

change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr]


datagen = ImageDataGenerator(horizontal_flip=True,width_shift_range=0.125,height_shift_range=0.125,fill_mode='constant',cval=0.)
datagen.fit(x_train)


model=build_model()
print(model.summary())
history=model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),steps_per_epoch=iterations,epochs=epochs, validation_data=(x_test, y_test),callbacks=cbks, shuffle=True)
# history=model.fit(x_train, y_train,batch_size=batch_size,steps_per_epoch=iterations,epochs=epochs, validation_data=(x_test, y_test),callbacks=cbks, shuffle=True)


model.save('cifar10_vgg.h5')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'val'])
plt.show()
plt.plot(history.history['val_accuracy'])
plt.legend([ 'val'])
plt.show()


2.5.0
170508288/170498071 [==============================] - 4s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 192)       14400     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 192)       36864     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 192)       36864     
_________________________________________________________________
average_pooling2d (AveragePo (None, 16, 16, 192)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 192)       921600    
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 192)       0         
___________

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200


KeyboardInterrupt: ignored